In [1]:
from dataloader import load_data_label_encoded
from trainer import train_model, evaluate_model
from snn import SNNModelSimple
import torch
import matplotlib.pyplot as plt
import pickle

Parameters that worked the best
1. Optimizer - Adam with lr ~ 0.005 and 0.001
2. Spike encoding ~ temporal with 50 time steps
3. Batch size - 128 (seems to work on-par with 32)
4. Beta - 0.8, 0.9, 0.999
5. Loss - CrossEntropyLoss
6. Num epochs - 100

In [ ]:
root_folder = "./tactile_dataset/"
file_name = "final_merged_df_sw500.csv"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_losses_beta = {}
train_accuracies_beta = {}
val_losses_beta = {}
val_accuracies_beta = {}
test_accuracies_beta = {}
for beta in [0.8]:
    print(f"Training for beta={beta}")
    train_loader, val_loader, test_loader, num_outputs, num_features = load_data_label_encoded(root_folder, file_name, spike_encoding='temporal', num_steps=50, batch_size=128, device=device)

    model = SNNModelSimple(num_features, num_outputs, beta=beta)
    model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

    train_losses, train_accuracies, val_losses, val_accuracies = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=50)
    train_losses_beta[beta] = train_losses
    train_accuracies_beta[beta] = train_accuracies
    val_losses_beta[beta] = val_losses
    val_accuracies_beta[beta] = val_accuracies

    test_accuracies_beta[beta] = evaluate_model(model, test_loader)

out_file_name = "exp_snn_beta_08.pkl"
with open(out_file_name, "wb") as f:
    pickle.dump({
        "train_losses": train_losses_beta,
        "train_accuracies": train_accuracies_beta,
        "val_losses": val_losses_beta,
        "val_accuracies": val_accuracies_beta,
        "test_accuracies": test_accuracies_beta
    }, f)